In [12]:
!pip install transformers


In [13]:
!pip install SpeechRecognition


In [14]:
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import geopy.distance
import schedule
import time
import requests

# Initialize text-to-speech engine
def initialize_text_to_speech():
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Adjust speaking rate
    engine.setProperty('voice', 'english')  # Set voice to English
    return engine

def speak(engine, text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def recognize_speech():
    """Recognize speech using the microphone."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            print("Processing...")
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "Sorry, I didn't understand that."
        except sr.RequestError:
            return "Sorry, there was an issue with the speech recognition service."
        except Exception as e:
            return str(e)

def get_current_time():
    """Return the current time."""
    now = datetime.now()
    return now.strftime("%I:%M %p")

def get_current_date_and_day():
    """Return the current date and day."""
    now = datetime.now()
    return now.strftime("%A, %B %d, %Y")

def get_location():
    """Get current location using geopy."""
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()
        return data.get('city', 'Unknown Location'), data.get('loc', '0,0')
    except Exception as e:
        return "Unknown Location", "0,0"

def calculate_distance_to_hospital():
    """Calculate the distance to the nearest hospital."""
    user_location = get_location()[1]
    hospital_location = "12.9715987,77.5945627"  # Example coordinates for a hospital
    try:
        user_coords = tuple(map(float, user_location.split(",")))
        hospital_coords = tuple(map(float, hospital_location.split(",")))
        distance = geopy.distance.distance(user_coords, hospital_coords).km
        return f"The hospital is approximately {distance:.2f} kilometers away."
    except Exception as e:
        return "Could not calculate the distance. Please ensure location services are enabled."

def set_daily_reminder(engine):
    """Set a daily reminder at 6 AM."""
    def reminder_task():
        speak(engine, "Good morning! It's 6 AM. Time to wake up!")

    schedule.every().day.at("06:00").do(reminder_task)
    return "Reminder set for 6 AM daily to wake you up."

def get_google_news():
    """Fetch Google News headlines using RSS feed."""
    try:
        news_feed_url = "https://news.google.com/rss"
        response = requests.get(news_feed_url)
        headlines = []
        if response.status_code == 200:
            from xml.etree import ElementTree as ET
            root = ET.fromstring(response.content)
            for item in root.findall(".//item")[:5]:
                headlines.append(item.find("title").text)
        return headlines
    except Exception as e:
        return ["An error occurred while fetching news headlines."]

def interpret_user_intent(user_input):
    """Interpret user input dynamically using keyword matching."""
    intents = {
        "time": ["time", "current time"],
        "date": ["date", "today"],
        "location": ["location", "where am I"],
        "distance": ["distance", "hospital"],
        "reminder": ["reminder", "wake up"],
        "news": ["news", "headlines"]
    }

    user_input_lower = user_input.lower()
    for intent, keywords in intents.items():
        if any(keyword in user_input_lower for keyword in keywords):
            return intent

    return "unknown"

def chatbot_response(engine, user_input):
    """Generate a response based on user intent."""
    intent = interpret_user_intent(user_input)

    if intent == "time":
        return f"The current time is {get_current_time()}."

    if intent == "date":
        return f"Today's date is {get_current_date_and_day()}."

    if intent == "location":
        location, _ = get_location()
        return f"You are currently in {location}."

    if intent == "distance":
        return calculate_distance_to_hospital()

    if intent == "reminder":
        return set_daily_reminder(engine)

    if intent == "news":
        headlines = get_google_news()
        return "Here are the top headlines: " + " | ".join(headlines)

    return "I'm not sure how to respond to that."

def main():
    engine = initialize_text_to_speech()

    speak(engine, "Hello! I am Lucky, your assistive chatbot. Say 'exit' to end the conversation.")

    while True:
        user_input = recognize_speech()
        print(f"You said: {user_input}")

        if user_input.lower() == "exit":
            speak(engine, "Goodbye! Have a nice day!")
            break

        response = chatbot_response(engine, user_input)
        print(f"Lucky: {response}")
        speak(engine, response)

        # Run pending reminders
        schedule.run_pending()
        time.sleep(1)

if __name__ == "__main__":
    main()


Listening...
Processing...
You said: Sorry, I didn't understand that.
Lucky: I'm not sure how to respond to that.
Listening...
Processing...
You said: the current time is
Lucky: The current time is 08:52 PM.
Listening...
Processing...
You said: what is today's date
Lucky: Today's date is Saturday, November 22, 2025.
Listening...
Processing...
You said: give me some top 5 news headlines
Lucky: Here are the top headlines: G20 Summit LIVE: PM Modi announces partnership with Canada, Australia, for innovation, technology - The Hindu | GRAP Rejig: Stage 4 Restrictions Shifted To Stage 3 As Delhi Pollution Deteriorates | Key Details - News18 | 'Will Rahul get message?': BJP mocks Congress as 'sore loser'; Shashi Tharoor's Trump-Mamdani remark spar - Times of India | India trade unions condemn new labour codes, plan nationwide protests - Reuters | Madhya Pradesh High Court temporarily stays demolition of Al-Falah University founder’s ancestral home - Bar and Bench
Listening...
Processing...
Yo

In [15]:
!pip install schedule

In [16]:
!pip install gradio pyttsx3 geopy requests
